In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Metin Madenciliği/train.csv') #Veriyi okuduk.
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df = df.dropna() # Boş satırları kaldırdık.
df.isnull().sum() # Boş satır var mı diye kontrol ettik.

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
X = df.drop(columns=['id','label'], axis=1) # Eğitim için kullanılacak featurelarımızı aldık. (title, author, text)
Y = df['label'] 
print(X.shape, Y.shape)


(18285, 3) (18285,)


In [ ]:
X.reset_index(inplace=True) # null değerler olduğu için index'i sıfırlayıp 0-18284 şeklinde indexledik.
X = X.drop('index',axis=1)
X

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
18280,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [ ]:
import nltk # Doğal dil işleme kütüphanesi
import re # Regular Expression noktalama işaretlerini ve rakamları atmak için kullanacağız.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer # Kelimlerin köküne inmek için kullanacağız.
from nltk.tokenize import word_tokenize

nltk.download('stopwords') # Veriden stopwordsleri kaldırmak için nltk kütüphanesini kullanacağız.
nltk.download('punkt') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # Kelime çantası elde etmek için kullanılacak kütüphane
from sklearn.linear_model import LogisticRegression # Modelimiz
from sklearn.model_selection import train_test_split # test ve train olarak ayırmak için kullandık.
from sklearn.metrics import confusion_matrix, classification_report # başarı metrikleri için kullanılır.


In [ ]:
def onIsleme(data):
  stemming = PorterStemmer()

  corpus = []
  for i in range(0,len(data)):

    regex = re.sub('[^a-zA-Z]',' ',data['title'][i]) # Tittle kısmından noktalama işaretlerini ve rakamları kaldırdık.

    regex = regex.lower() #Tüm harfleri küçük harfe çevirdik
    regex = regex.split() 

    regex = [stemming.stem(word) for word in regex if not word in stopwords.words('english')] #stopwordsler çıkarıldı ve kelimelerin köküne inildi.

    regex = ' '.join(regex)
    # Önce kelimelerin köküne inildi daha sonra kelimenin stopwords olup olmadığına bakıldı. Her satırdaki kelime için bu işlem yapıldı ve corpus'a eklendi.
    corpus.append(regex)

  print(corpus[:3])
  return corpus

In [ ]:
corpus = onIsleme(X)

['hous dem aid even see comey letter jason chaffetz tweet', 'flynn hillari clinton big woman campu breitbart', 'truth might get fire']


In [ ]:
def donusturme(data):
  vect = CountVectorizer()
  vect.fit_transform(data) # Kelimeri sayısallaştırdık. Her kelimeye alfabetik olarak bir sayı atandı.
  
  print('Vocabulary size: {0}'.format(len(vect.vocabulary_))) # Toplam kelime sayısı
  print('Vocabulary content: {0}'.format(vect.vocabulary_)) # Kelimeler ve atanan sayılar

  bag_of_words = vect.transform(data)
  print(bag_of_words[0]) # İlk satırdaki kelime ile oluşan matrisin ilk satırı

  print('bag of words matris: \n {}'.format(bag_of_words.toarray())) # Matris olarak gösterimi

  return bag_of_words

In [ ]:
bag_of_words = donusturme(corpus)

Vocabulary size: 13913
Vocabulary content: {'hous': 5763, 'dem': 3128, 'aid': 225, 'even': 4104, 'see': 10930, 'comey': 2405, 'letter': 7007, 'jason': 6335, 'chaffetz': 2017, 'tweet': 12751, 'flynn': 4562, 'hillari': 5604, 'clinton': 2277, 'big': 1218, 'woman': 13672, 'campu': 1810, 'breitbart': 1537, 'truth': 12690, 'might': 7765, 'get': 4947, 'fire': 4462, 'civilian': 2218, 'kill': 6639, 'singl': 11252, 'us': 13066, 'airstrik': 247, 'identifi': 5881, 'iranian': 6226, 'jail': 6307, 'fiction': 4414, 'unpublish': 12973, 'stori': 11811, 'stone': 11799, 'death': 3009, 'adulteri': 144, 'jacki': 6295, 'mason': 7502, 'hollywood': 5678, 'would': 13712, 'love': 7208, 'trump': 12665, 'bomb': 1409, 'north': 8436, 'korea': 6735, 'lack': 6799, 'tran': 12562, 'bathroom': 1029, 'exclus': 4143, 'video': 13215, 'beno': 1151, 'hamon': 5348, 'win': 13616, 'french': 4717, 'socialist': 11426, 'parti': 8968, 'presidenti': 9557, 'nomin': 8415, 'new': 8316, 'york': 13821, 'time': 12418, 'back': 871, 'channel

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(bag_of_words.toarray(), Y, test_size = 0.2, random_state=58)
#%70 train %30 test olarak ayırdık.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

model = Sequential(name='MyModel')

model.add(Dense(64, activation='relu',kernel_regularizer='l2', input_shape=(None, 14628, 13913)))
model.add(Dense(128, activation='relu',kernel_regularizer='l2'))
model.add(Dense(64, activation='relu',kernel_regularizer='l2'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.fit(Xtrain, Ytrain, epochs=20, validation_split=0.2, batch_size=1024)

Epoch 1/20
12/12 [==============================] - 2s 115ms/step - loss: 2.8633 - acc: 0.5652 - val_loss: 2.1730 - val_acc: 0.5759
Epoch 2/20
12/12 [==============================] - 1s 64ms/step - loss: 1.9531 - acc: 0.6179 - val_loss: 1.7429 - val_acc: 0.7109
Epoch 3/20
12/12 [==============================] - 1s 64ms/step - loss: 1.6056 - acc: 0.7725 - val_loss: 1.4300 - val_acc: 0.8414
Epoch 4/20
12/12 [==============================] - 1s 65ms/step - loss: 1.2960 - acc: 0.9064 - val_loss: 1.1556 - val_acc: 0.9077
Epoch 5/20
12/12 [==============================] - 1s 64ms/step - loss: 1.0501 - acc: 0.9332 - val_loss: 0.9571 - val_acc: 0.9057
Epoch 6/20
12/12 [==============================] - 1s 65ms/step - loss: 0.8701 - acc: 0.9304 - val_loss: 0.8124 - val_acc: 0.9115
Epoch 7/20
12/12 [==============================] - 1s 62ms/step - loss: 0.7370 - acc: 0.9340 - val_loss: 0.7048 - val_acc: 0.9187
Epoch 8/20
12/12 [==============================] - 1s 62ms/step - loss: 0.6366 - 

In [ ]:
test_acc, test_loss = model.evaluate(Xtest, Ytest)
print('Test doğruluğu {} loss değeri  {}'.format(test_loss,test_acc))

115/115 [==============================] - 1s 7ms/step - loss: 0.3135 - acc: 0.9284
Test doğruluğu 0.9283565878868103 loss değeri  0.3134634792804718


In [ ]:
Ypred = model.predict(Xtest)

confusion = confusion_matrix(Ytest,np.round(Ypred),labels=[0,1])
tn, fp, fn, tp = confusion_matrix(Ytest,np.round(Ypred),labels=[0,1]).ravel()

print('Confusion matrix: \n {}'.format(confusion))
print('\n')
print('TP: {} FP: {} TN: {} FN:  {}'.format(tp,fp,tn,fn))
print('\n')
print(classification_report(Ytest,np.round(Ypred)))

Confusion matrix: 
 [[1881  213]
 [  49 1514]]


TP: 1514 FP: 213 TN: 1881 FN:  49


              precision    recall  f1-score   support

           0       0.97      0.90      0.93      2094
           1       0.88      0.97      0.92      1563

    accuracy                           0.93      3657
   macro avg       0.93      0.93      0.93      3657
weighted avg       0.93      0.93      0.93      3657

